In [1]:
from transformers import pipeline
# from transformers import BertTokenizer, TFBertModel
# import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.preprocessing import LabelBinarizer
import seaborn as sns
import os

In [ ]:
datapath = "../data/"
X_train = pd.read_csv(os.path.join(datapath, "challenge4/train.txt"))
X_test = pd.read_csv(os.path.join(datapath, "challenge4/test.txt"))